In [162]:
import pandas as pd
import numpy as np

In [163]:
df = pd.read_csv("audio_allp_raw.csv")

In [164]:
# create unique id for each sentence

df['sentence_uniq_id'] = df['doc_id'].apply(str) + df['paragraph_id'].apply(str) + df['sentence_id'].apply(str)


In [165]:
# identify last PUNCT on every sentence

df["end_of_sentence"] = False
df["token_length"] = 0

current_id = df.loc[0,'sentence_uniq_id']
for index, row in df.iterrows():
    if row["sentence_uniq_id"] != current_id:
        current_id = row["sentence_uniq_id"]
        df.at[index-1, "end_of_sentence"] = True
        df.at[index-1, "token_length"] = df.at[index-1, "token_id"] - 1

#cast token_length to string
df.token_length = pd.to_numeric(df.token_length, errors='coerce')

In [166]:
# calculate dependency_length for each token

df['dependency_length'] = df['token_id'] - df['head_token_id']

In [167]:
# calculate jumlah dependency_lenght for each sentence

current_id = df.loc[0,'sentence_uniq_id']
sum_dependency_length = 0
df["sum_dependency_length"] = np.NaN

for index, row in df.iterrows():
    if row["sentence_uniq_id"] == current_id:
        if not row.end_of_sentence: # not counting the dependency_length of last PUNCT
            sum_dependency_length += abs(row["dependency_length"])
    else:
        df.at[index-1, "sum_dependency_length"] = sum_dependency_length
        sum_dependency_length = 0
        current_id = row["sentence_uniq_id"]
        

In [168]:
# average maju dan average mundur per kalimat

# initialize value for average positive and negative value for each sentence
df["avg_positive_dependency"] = None
df["avg_negative_dependency"] = None

# initialize value for count and sum
list_positive_dependency = []
list_negative_dependency = []

# init current_sentence_id with first sentence_uniq_id
current_sentence_id = df.loc[0,'sentence_uniq_id']

for index, row in df.iterrows():
    if row["sentence_uniq_id"] == current_sentence_id:
        # add element to list negative dependency
        if row.dependency_length < 0:
            list_negative_dependency.append(row.dependency_length)
        else:
            list_positive_dependency.append(row.dependency_length)
    else:
        # write averera dependency on data frame
        df.at[index-1, "avg_positive_dependency"] = np.mean(list_positive_dependency)
        df.at[index-1, "avg_negative_dependency"] = np.abs(np.mean(list_negative_dependency))
        current_sentence_id = row["sentence_uniq_id"]


In [169]:
import plotly.plotly as py
import cufflinks as cf

In [170]:
#Find Aveverage Dependency Length
df["avg_dependency_length"] = df.sum_dependency_length / (df.token_length - 1)


In [172]:
from plotly.graph_objs import *

py.iplot({
    'data': [
        Bar(**{
            'x': df.loc[df.sum_dependency_length.notnull(),"token_length"], 
            'y': df.loc[df.sum_dependency_length.notnull(),"avg_dependency_length"],
            'name' : 'AVG Dependency Length',
            'type': 'bar'
        })
    ],
    'layout': Layout(**{
        'title': 'AVG Dependency Length'
    })
})

In [ ]:
from plotly.graph_objs import *

py.iplot({
    'data': [
        Bar(**{
            'x': df.loc[df.sum_dependency_length.notnull(),"token_length"], 
            'y': df.loc[df.sum_dependency_length.notnull(),"avg_dependency_length"],
            'name' : 'AVG Dependency Length',
            'type': 'bar'
        })
    ],
    'layout': Layout(**{
        'title': 'AVG Dependency Length'
    })
})

In [ ]:
from plotly.graph_objs import *
df = df.loc[df.token_length.isin(['5','6','7','8','9'])]
py.iplot({
    'data': [
        Bar(**{
            'x': df.loc[df.sum_dependency_length.notnull(),"token_length"], 
            'y': df.loc[df.sum_dependency_length.notnull(),"avg_dependency_length"],
            'name' : 'AVG Dependency Length',
            'type': 'bar'
        })
    ],
    'layout': Layout(**{
        'title': 'AVG Dependency Length'
    })
})

In [ ]:



#liat jumlah sentence di setiap cluster
#liat average dari average jarak dependensi di masing2 cluster
#liat average maju dan mundur di masing masing cluster
#bikin histogram 8 dan 9
#bikin algo untuk ubah posisi kata ATAU
#bikin algo untuk ubah posisi root ATAU
#ngitung average dependency dengan nge-random depedendency root nya



